In [1]:
import uvicorn 
import gensim
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from fastapi import FastAPI
from fastapi import Header
from pydantic import BaseModel
import pandas as pd 
import numpy as np 
from gensim.test.utils import datapath
from ast import literal_eval
from gensim import corpora
import os
#from data_ingestion import read_txt_content, preprocess_data, get_date, get_ref, get_files_list
#from kpi import load_corpus_model, preprocess_data, load_lda_model, calculate_coherence
import pandas as pd 

In [2]:
folder_name = 'CI_newspaper_subcorpora'
pub_refs = ["2012271201", "sn85054967", "sn93053873", "sn85066408", "sn85055164", "sn84037024", "sn84037025", "sn84020351", "sn86092310", "sn92051386"]
pub_names = ["Cronaca_Sovversiva", "Il_Patriota", "L'Indipendente", "L'Italia", "La_Libera_Parola", "La_Ragione", "La_Rassegna", "La_Sentinella", "La_Sentinella_del_West", "La_Tribuna_del_Connecticut"]
date_ref_1 = "1903-06-06"
date_ref_2 = "1919-05-01"

In [3]:
corpus_model_file = 'corpus_model.csv'
current_directory = os.getcwd()
lda_model_file = os.path.join(current_directory, 'lda_model')  # Construct the path to the LDA model file
bigrams = load_corpus_model(corpus_model_file)
id2word, corpus = preprocess_data(bigrams)
lda = load_lda_model(lda_model_file)

In [4]:
topic_print_model = lda.print_topics(num_words=30)

In [6]:
df_topic_print_model = pd.DataFrame(topic_print_model, columns=['topic_id','words'])

In [11]:
topic_print_model

[(0,
  '0.001*"pace" + 0.001*"america" + 0.001*"francisco" + 0.001*"pubblico" + 0.001*"loggia" + 0.001*"sera" + 0.001*"vero" + 0.001*"patria" + 0.001*"morte" + 0.001*"capo" + 0.001*"alia" + 0.001*"figli" + 0.001*"società" + 0.001*"venne" + 0.001*"legge" + 0.001*"dollari" + 0.001*"colonia" + 0.001*"vittoria" + 0.001*"comitato" + 0.001*"londra" + 0.001*"cuore" + 0.001*"storia" + 0.001*"disse" + 0.001*"diritto" + 0.001*"fatti" + 0.001*"direttore" + 0.001*"altra" + 0.001*"delia" + 0.001*"milano" + 0.001*"lire"'),
 (1,
  '0.002*"pace" + 0.002*"austriaci" + 0.002*"tedeschi" + 0.001*"wilson" + 0.001*"alleati" + 0.001*"nemico" + 0.001*"congresso" + 0.001*"vittoria" + 0.001*"nazioni" + 0.001*"questione" + 0.001*"america" + 0.001*"londra" + 0.001*"politica" + 0.001*"americani" + 0.001*"ltalia" + 0.001*"diritto" + 0.001*"americano" + 0.001*"legge" + 0.001*"marzo" + 0.001*"ufficiali" + 0.001*"patria" + 0.001*"francese" + 0.001*"italiane" + 0.001*"presidente_wilson" + 0.001*"lotta" + 0.001*"militar

In [20]:
print(topic_print_model[0][0], topic_print_model[0][1])


0 0.001*"pace" + 0.001*"america" + 0.001*"francisco" + 0.001*"pubblico" + 0.001*"loggia" + 0.001*"sera" + 0.001*"vero" + 0.001*"patria" + 0.001*"morte" + 0.001*"capo" + 0.001*"alia" + 0.001*"figli" + 0.001*"società" + 0.001*"venne" + 0.001*"legge" + 0.001*"dollari" + 0.001*"colonia" + 0.001*"vittoria" + 0.001*"comitato" + 0.001*"londra" + 0.001*"cuore" + 0.001*"storia" + 0.001*"disse" + 0.001*"diritto" + 0.001*"fatti" + 0.001*"direttore" + 0.001*"altra" + 0.001*"delia" + 0.001*"milano" + 0.001*"lire"


In [25]:
dic = { topic_print_model[i][0] : topic_print_model[i][1]  for i in range (7)}

In [26]:
dic

{0: '0.001*"pace" + 0.001*"america" + 0.001*"francisco" + 0.001*"pubblico" + 0.001*"loggia" + 0.001*"sera" + 0.001*"vero" + 0.001*"patria" + 0.001*"morte" + 0.001*"capo" + 0.001*"alia" + 0.001*"figli" + 0.001*"società" + 0.001*"venne" + 0.001*"legge" + 0.001*"dollari" + 0.001*"colonia" + 0.001*"vittoria" + 0.001*"comitato" + 0.001*"londra" + 0.001*"cuore" + 0.001*"storia" + 0.001*"disse" + 0.001*"diritto" + 0.001*"fatti" + 0.001*"direttore" + 0.001*"altra" + 0.001*"delia" + 0.001*"milano" + 0.001*"lire"',
 1: '0.002*"pace" + 0.002*"austriaci" + 0.002*"tedeschi" + 0.001*"wilson" + 0.001*"alleati" + 0.001*"nemico" + 0.001*"congresso" + 0.001*"vittoria" + 0.001*"nazioni" + 0.001*"questione" + 0.001*"america" + 0.001*"londra" + 0.001*"politica" + 0.001*"americani" + 0.001*"ltalia" + 0.001*"diritto" + 0.001*"americano" + 0.001*"legge" + 0.001*"marzo" + 0.001*"ufficiali" + 0.001*"patria" + 0.001*"francese" + 0.001*"italiane" + 0.001*"presidente_wilson" + 0.001*"lotta" + 0.001*"militare" + 0.

In [4]:
import requests

def test_api():
    response = requests.get('http://localhost:8000/')
    return response

a =test_api()

In [11]:
a.json()

"Hello, I'm working"

In [62]:
from requests.auth import HTTPBasicAuth

data = {
  "num_topic": 2,
  "date_ref_1": "1940-06-06",
  "date_ref_2": "1941-05-01"
}

b = requests.post('http://localhost:8000/topic', json=data, auth=HTTPBasicAuth('fdmin', 'mdp'))


In [73]:
list(b.json().values())[0]


'Incorrect email or password'

In [75]:
list(b.json().values())[0] == 'Incorrect email or password'


True

In [6]:
pd.read_csv('subset.csv').head()

,Unnamed: 0,file_name,date,publication_ref,publication_name,file_content,chars_count,words_count
0,0,/La_Sentinella/sn84020351_1930-06-07_ed-1_seq-...,1930-06-07,sn84020351,La_Sentinella,"C'è ancora da combattere da soffrire, da vinc...",20790,3609
1,1,/La_Sentinella/sn84020351_1930-06-14_ed-1_seq-...,1930-06-14,sn84020351,La_Sentinella,"C'è ancora da combattere da soffrire, da vinc...",21810,3779
2,2,/La_Sentinella/sn84020351_1930-06-21_ed-1_seq-...,1930-06-21,sn84020351,La_Sentinella,MICHAEL F. CHINCO MORRIS SILVERBURY BENCH. Il...,20844,3571
3,3,/La_Sentinella/sn84020351_1930-06-28_ed-1_seq-...,1930-06-28,sn84020351,La_Sentinella,4 LUGLIO DUE ANNIVERSARII FATIDICI. Venerdì p...,19681,3351
4,4,/La_Sentinella/sn84020351_1930-07-05_ed-1_seq-...,1930-07-05,sn84020351,La_Sentinella,". C'è ancora da combattere da soffrire, da vi...",20463,3460


In [21]:
from pre_processing import load_data
corpus_df = load_data('subset.csv')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Florian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Florian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
corpus_df

,Unnamed: 0,file_name,date,publication_ref,publication_name,file_content,chars_count,words_count
0,0,/La_Sentinella/sn84020351_1930-06-07_ed-1_seq-...,1930-06-07,sn84020351,La_Sentinella,"C'è ancora da combattere da soffrire, da vinc...",20790,3609
1,1,/La_Sentinella/sn84020351_1930-06-14_ed-1_seq-...,1930-06-14,sn84020351,La_Sentinella,"C'è ancora da combattere da soffrire, da vinc...",21810,3779
2,2,/La_Sentinella/sn84020351_1930-06-21_ed-1_seq-...,1930-06-21,sn84020351,La_Sentinella,MICHAEL F. CHINCO MORRIS SILVERBURY BENCH. Il...,20844,3571
3,3,/La_Sentinella/sn84020351_1930-06-28_ed-1_seq-...,1930-06-28,sn84020351,La_Sentinella,4 LUGLIO DUE ANNIVERSARII FATIDICI. Venerdì p...,19681,3351
4,4,/La_Sentinella/sn84020351_1930-07-05_ed-1_seq-...,1930-07-05,sn84020351,La_Sentinella,". C'è ancora da combattere da soffrire, da vi...",20463,3460
5,5,/La_Sentinella/sn84020351_1930-07-12_ed-1_seq-...,1930-07-12,sn84020351,La_Sentinella,"C'è ancora da combattere da soffrire, da vinc...",21248,3608
6,6,/La_Sentinella/sn84020351_1930-07-19_ed-1_seq-...,1930-07-19,sn84020351,La_Sentinella,"C'è ancora da combattere da soffrire, da vinc...",19055,3191
7,7,/La_Sentinella/sn84020351_1930-07-26_ed-1_seq-...,1930-07-26,sn84020351,La_Sentinella,"C'è àncora da combattere da soffrire, da vinc...",21385,3757
8,8,/La_Sentinella/sn84020351_1930-08-02_ed-1_seq-...,1930-08-02,sn84020351,La_Sentinella,"C'è ancora da combattere da soffrire, da vinc...",20110,3305
9,9,/La_Sentinella/sn84020351_1930-08-09_ed-1_seq-...,1930-08-09,sn84020351,La_Sentinella,"BRIDGEPORT, CONN,. SABATO- 9 AGOSTO 1930 jncr...",20700,3512


In [3]:
from sqlalchemy import URL, create_engine, inspect, text

url_object = URL.create(
    "mysql+pymysql",
    username="root",
    password="password",
    host="localhost",
    database="DB",
)

engine = create_engine(url_object)
inspector = inspect(engine)

In [18]:
date_ref_1 = "1903-6-6"
date_ref_2 = "1903-6-29"
query = text(f"SELECT * FROM sources WHERE date <= DATE '{date_ref_2}' AND date > DATE '{date_ref_1}'")
with engine.connect() as conn:
    result = conn.execute(query)
    for row in result:
        print(row)

(1, 'Cronaca_Sovversiva\\2012271201_1903-06-13_ed-1_seq-1_ocr.txt', '1903-06-13', '2012271201', 'Cronaca_Sovversiva', ' Ebdomadario anarchico di propaganda rivoluzionaria. XJt redezt miserìs abeat fortuna, superbis! i Interno per I\' estero, $1.00 " " .50 manoscritti  ... (11416 characters truncated) ... llitet, yzione. " non ne devono vedere altri. " (1) Doveva mandarci l\'artico !o commemoratiro del ibo pane e preferivano andarsene piuttosto che i. ')
(2, 'Cronaca_Sovversiva\\2012271201_1903-06-20_ed-1_seq-1_ocr.txt', '1903-06-20', '2012271201', 'Cronaca_Sovversiva', ' ANNO I. NUMERO 31 VVER Ebdomadario anarchico di propaganda rivoluzionaria. Ut redeat miserìs abeat fortuna superbis! CRONACA Abbonamento annuo per I ... (11169 characters truncated) ...  ma (Dal Lu- Tieni i) LA " PROTESTA UMANA" avverte lettori e compagni che\' il su nuovo recapito c7 Sad Franisco, California P O .Box 2(uS- f " i fi ')
(3, 'Cronaca_Sovversiva\\2012271201_1903-06-27_ed-1_seq-1_ocr.txt', '1903-06-27', '2

In [24]:
df = pd.read_sql_query(query, engine)

In [25]:
df

,index,file_name,date,publication_ref,publication_name,file_content
0,1,Cronaca_Sovversiva\2012271201_1903-06-13_ed-1_...,1903-06-13,2012271201,Cronaca_Sovversiva,Ebdomadario anarchico di propaganda rivoluzio...
1,2,Cronaca_Sovversiva\2012271201_1903-06-20_ed-1_...,1903-06-20,2012271201,Cronaca_Sovversiva,ANNO I. NUMERO 31 VVER Ebdomadario anarchico ...
2,3,Cronaca_Sovversiva\2012271201_1903-06-27_ed-1_...,1903-06-27,2012271201,Cronaca_Sovversiva,Ebdomadàrio anarchico di propaganda rivoluzio...
3,2194,L'Italia\sn85066408_1903-06-08_ed-1_seq-1_ocr.txt,1903-06-08,sn85066408,L'Italia,ANNO XY11 GIORNALE ITALIANO PATTO PER GL’ITAL...
4,2195,L'Italia\sn85066408_1903-06-09_ed-1_seq-1_ocr.txt,1903-06-09,sn85066408,L'Italia,ANNO XY11 «(ORNALE ITALIANO FATTO PER GL’ITAL...
5,2196,L'Italia\sn85066408_1903-06-10_ed-1_seq-1_ocr.txt,1903-06-10,sn85066408,L'Italia,ANNO xyu «•ONNAUK ITALIANO PATTO MR — i i PHO...
6,2197,L'Italia\sn85066408_1903-06-11_ed-1_seq-1_ocr.txt,1903-06-11,sn85066408,L'Italia,AN3TO XVU GIORNALE ITALIANO FATTO FER GL'ITAL...
7,2198,L'Italia\sn85066408_1903-06-12_ed-1_seq-1_ocr.txt,1903-06-12,sn85066408,L'Italia,AXNO XVU GIORNALE ITALIANO FATTO F*g OL'ITAUA...
8,2199,L'Italia\sn85066408_1903-06-13_ed-1_seq-1_ocr.txt,1903-06-13,sn85066408,L'Italia,me xvu ClOFMALM rfftUANO FATTO SL'ITAUANt THE...
9,2200,L'Italia\sn85066408_1903-06-15_ed-1_seq-1_ocr.txt,1903-06-15,sn85066408,L'Italia,ANNO XVU GIORNALE ITALIANO FATTO PER GL'ITALI...


In [4]:
query = text("SELECT * FROM sources ")
with engine.connect() as conn:
    result = conn.execute(query)
    liss=[]
    for row in result:
        liss.append(row)
    print(liss[-1])

IndexError: list index out of range

In [65]:
liss[-1][0]

8652

In [91]:
liss[-1]

(8652, 'La_Tribuna_del_Connecticut\\sn92051386_1908-12-09_ed-1_seq-1_ocr.txt', '1908-12-09', 'sn92051386', 'La_Tribuna_del_Connecticut', ' Frasi d\'avvoccaf o Àllà Camera francese si ri prende, di tanto in tanto, la discussione sulla pena di mor te, che alcuni vogliono abolita altri man ... (9819 characters truncated) ... TACV3PAT 5y - il 0 rai 5 5; "j. "V "V V- 1 111 I, r ; l felli A ESTÉSI if i F! h ffii a - n nn n fp n p ppph n ran M S\'. vM 0 1 000yy EASE IN suo 0 ')

In [5]:
query = text("SELECT * FROM sources WHERE date ='1917-04-25'")
with engine.connect() as conn:
    result = conn.execute(query)
lis=[]
for row in result:
    lis.append(row)


In [6]:
lis

[]

In [134]:
query = text("INSERT INTO sources (file_name,file_content,date,publication_name,publication_ref) VALUES ('La_Tribuna_del_Connecticut\\sn92051386_1908-12-20_ed-1_seq-1_ocr.txt','something','1908-12-20','La_Tribuna_del_Connecticut','sn92051386')")
    
with engine.connect() as connection: 
    result = connection.execute(query)
    


ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [153]:
query = text('SELECT COUNT(*) FROM sources')
with engine.connect() as conn:
    result = conn.execute(query)
    for row in result:
        print(row)

(8655,)


In [110]:
with engine.connect() as connection:
     connection.execute(query) 
     last_insert_id = connection.execute("SELECT LAST_INSERT_ID").fetchone()[0] 
 

ObjectNotExecutableError: Not an executable object: 'SELECT LAST_INSERT_ID'

In [135]:
query = text("INSERT INTO sources (file_name,file_content,date,publication_name,publication_ref) VALUES ('a','a','a','a','a')")
    
with engine.connect() as connection: 
    result = connection.execute(query)
    


In [150]:
file_name = 'b'
file_content= 'b'
date= 'b'
publication_name= 'b'
publication_ref= 'b'

In [158]:
import mysql.connector

#query = text("INSERT INTO sources (file_name,file_content,date,publication_name,publication_ref) VALUES ('a','a','a','a','a')")
connection = mysql.connector.connect(username="root",
    password="password",
    host="localhost",
    database="DB"
    )
query = f"""INSERT INTO sources (file_name,file_content,date,publication_name,publication_ref) VALUES ('{file_name}','{file_content}','{date}','{publication_name}','{publication_ref}')"""
cursor = connection.cursor()
cursor.execute(query)
connection.commit()
cursor.close()
print(cursor.rowcount, "Record inserted successfully into Laptop table")


1 Record inserted successfully into Laptop table


#Syntaxe correcte pour ajouter des données dans database 

In [ ]:
import mysql.connector

try:
    connection = mysql.connector.connect(host='localhost',
                                         database='electronics',
                                         user='pynative',
                                         password='pynative@#29')

    mySql_insert_query = """INSERT INTO Laptop (Id, Name, Price, Purchase_date) 
                           VALUES 
                           (15, 'Lenovo ThinkPad P71', 6459, '2019-08-14') """

    cursor = connection.cursor()
    cursor.execute(mySql_insert_query)
    connection.commit()
    print(cursor.rowcount, "Record inserted successfully into Laptop table")
    cursor.close()

except mysql.connector.Error as error:
    print("Failed to insert record into Laptop table {}".format(error))

finally:
    if connection.is_connected():
        connection.close()
        print("MySQL connection is closed")

In [204]:
date = '1930-12-20'

In [208]:
query = text(f"SELECT * FROM sources WHERE date ='{date}'")
with engine.connect() as conn:
    result = conn.execute(query)
lis=[]
for row in result:
    lis.append(row)
lis

[(8469, 'La_Sentinella\\sn84020351_1930-12-20_ed-1_seq-1_ocr.txt', '1930-12-20', 'sn84020351', 'La_Sentinella', ' C\'è ancora da combattere da soffrire, da vincere. CARDUCCI. Muoviamoci! Notiamo con vero rammarico che nella terri- bile crisi economica che attrav ... (20391 characters truncated) ... r. Rogers forgot that two years 7. o v ?,go. they betl?yed Judge Pallotti and this, time got a dose of (Continua a pag. 2.a) , their own medicine. , ')]

In [200]:
len(lis)
len(lis[0])

6

In [215]:
lis[0][1]

'La_Sentinella\\sn84020351_1930-12-20_ed-1_seq-1_ocr.txt'

In [218]:
text=[]
file_name =[]
for i in range (len(lis)):    
    text.append(lis[i][5])
for i in range (len(lis)): 
    file_name.append(lis[i][1])
dic = { file_name[i] : text[i]  for i in range (len(lis))}

In [219]:
dic

{'La_Sentinella\\sn84020351_1930-12-20_ed-1_seq-1_ocr.txt': ' C\'è ancora da combattere da soffrire, da vincere. CARDUCCI. Muoviamoci! Notiamo con vero rammarico che nella terri- bile crisi economica che attraversiamo, la nostra Colonia, come ente, non s\'è dimostrata, nè si di mostra all\'altezza della situazione. Mentre nelle altre comunità, i nostri conna zionali, più o meno benestanti, organizzano comi- jn 4?,-i; a; v,n.ini; o. gnore, e quel giorno brillerà di! ARTICOLO DEL REV. PROF. E. IASIELLO "Il venturo Re venite, adoriamo! Rallegratevi, o figlie di Sion; esulta di gioia, o progenie di pienezza Cosi il verbo scendeva dal , Cielo, e per Maria, vergine puris sima d\'ogni grazia eletta, nasce va ignorato, povero, derelitto rrandissima luce, stilleranno dol- I nella solitaria canarina il Reden- mente risentono i disastrosi effetti della disOCCU- cezza i monti, e dai colli fluirà j tore del mondo. Cosi rinnovavasi l\'altissimo prodigio dell\'onnipotenza divina in un singolare, ammi

In [186]:
objet = ['file_name','file_name','file_name','file_name','file_name']

In [190]:
dict(zip(objet,lis))

TypeError: 'dict' object is not callable